In [5]:
CRITERIA = {
    "Squat": [
        {
            "name": "knees",
            "description": "Knees reach 90 degrees",
            "long_description": "The angle between your hip, knee, and ankle must reach 90 degrees. Your thigh should be parallel to the ground.",
            "condition": lambda angles: angles["right_hipright_knee"] <= 90,
            "requirement": "require",
            "feedback_fail": "Lower your squat to at least 90 degrees!",
            "feedback_pass": "Great depth! Keep it up!",
        },
        {
            "name": "shoulders",
            "description": "Shoulders remain level",
            "long_description": "Your shoulders should be at the same height during the squat to avoid tilting.",
            "condition": lambda angles: angles["left_shoulderright_shoulder"] < 10,
            "requirement": "avoid",
            "feedback_fail": "Keep your shoulders level while squatting!",
            "feedback_pass": "Nice! Your shoulders stayed balanced!",
        },
        {
            "name": "torso",
            "description": "Torso remains upright",
            "long_description": "Your back should be straight, and eyes forward.",
            "condition": lambda angles: angles["right_shoulderright_hip"] >= 70,
            "requirement": "avoid",
            "feedback_fail": "Straighten your back and look forward!",
            "feedback_pass": "Great posture!",
        },
    ],
    "Push-Up": [
        {
            "name": "level-shoulders",
            "description": "Shoulders stay level",
            "long_description": "Your shoulders should not tilt past 25 degrees.",
            "condition": lambda angles: angles["left_shoulderright_shoulder"] < 25,
            "requirement": "avoid",
            "feedback_fail": "Keep your shoulders steady during push-ups!",
            "feedback_pass": "Good control over your shoulders!",
        },
        {
            "name": "tricep",
            "description": "Arms fully extend",
            "long_description": "At the top position, your triceps should form a 90-degree angle with the ground.",
            "condition": lambda angles: angles["right_shoulderright_elbow"] >= 80,
            "requirement": "require",
            "feedback_fail": "Extend your arms fully at the top!",
            "feedback_pass": "Good full extension!",
        },
    ],
    "Sit-Up": [
        {
            "name": "up-position",
            "description": "Back gets perpendicular to ground",
            "long_description": "When you are in the 'up' position, your back should be at least 75 degrees.",
            "condition": lambda angles: angles["right_shoulderright_hip"] >= 75,
            "requirement": "require",
            "feedback_fail": "Sit up higher to reach the correct position!",
            "feedback_pass": "Good sit-up position!",
        },
        {
            "name": "down-position",
            "description": "Back gets horizontal to ground",
            "long_description": "When you are in the down position, your back should be flat with the ground, including your neck.",
            "condition": lambda angles: angles["left_shoulderleft_hip"] <= 5,
            "requirement": "require",
            "feedback_fail": "Lie back fully to complete the rep!",
            "feedback_pass": "Nice full range of motion!",
        },
        {
            "name": "bent legs",
            "description": "Legs stay bent",
            "long_description": "During a sit-up, your legs should stay bent and not move excessively.",
            "condition": lambda angles: angles["right_hipright_knee"] <= 10,
            "requirement": "avoid",
            "feedback_fail": "Keep your legs stable and bent throughout the movement!",
            "feedback_pass": "Great job keeping your legs steady!",
        },
    ]
}


In [6]:
import cv2
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub

import kagglehub as kh


# Load the MoveNet model
#model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")

# Download the latest version of the model.
path = kh.model_download("google/movenet/tensorFlow2/singlepose-lightning")
print("Path to model files:", path)

# Load the model from the local path.
model = tf.saved_model.load(path)
movenet = model.signatures["serving_default"]


def process_frame(frame):
    # Resize frame to 192x192 (assuming MoveNet is using this size)
    input_tensor = tf.image.resize(frame, (192, 192))
    
    # Ensure the tensor is int32 instead of float32
    input_tensor = tf.cast(input_tensor, dtype=tf.int32)

    # Add batch dimension
    input_tensor = tf.expand_dims(input_tensor, axis=0)

    # Run inference
    outputs = model.signatures["serving_default"](input_tensor)

    # Extract keypoints from output tensor
    keypoints = outputs["output_0"].numpy()  # Shape: (1, 1, 17, 3)

    return keypoints
# Open video capture
video_path = "squat-exercise.mp4"
cap = cv2.VideoCapture(0)  # Change to file path if using a video file

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    keypoints = process_frame(frame)
    #calculate_squat(keypoints)
      

    cv2.imshow("Pose Estimation", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


/Users/ethankondev/Documents/COSI142/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to model files: /Users/ethankondev/.cache/kagglehub/models/google/movenet/tensorFlow2/singlepose-lightning/4


2025-03-13 15:58:45.445 Python[97662:1119432] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2025-03-13 15:58:46.732 Python[97662:1119432] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-13 15:58:46.732 Python[97662:1119432] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## Displaying keypoints 

In [1]:
import cv2
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import kagglehub as kh

# Download the latest version of the model.
path = kh.model_download("google/movenet/tensorFlow2/singlepose-lightning")
print("Path to model files:", path)

# Load the model from the local path.
model = tf.saved_model.load(path)
# Although movenet is set, we'll use model.signatures["serving_default"] directly.
movenet = model.signatures["serving_default"]

def process_frame(frame):
    # Resize frame to 192x192 (assuming MoveNet is using this size)
    input_tensor = tf.image.resize(frame, (192, 192))
    # Convert to int32 as expected by the model
    input_tensor = tf.cast(input_tensor, dtype=tf.int32)
    # Add batch dimension
    input_tensor = tf.expand_dims(input_tensor, axis=0)
    # Run inference
    outputs = model.signatures["serving_default"](input_tensor)
    # Extract keypoints from output tensor (shape: [1, 1, 17, 3])
    keypoints = outputs["output_0"].numpy()
    return keypoints

def draw_keypoints(frame, keypoints, threshold=0.3):
    """
    Draws circles on the frame for each keypoint with a confidence above the threshold.
    Assumes keypoints are in normalized coordinates.
    """
    # Get frame dimensions
    height, width, _ = frame.shape
    # keypoints shape is (1, 1, 17, 3). Squeeze to get shape (17, 3)
    kpts = keypoints[0, 0, :, :]
    for kp in kpts:
        y, x, conf = kp
        if conf > threshold:
            # Scale coordinates to the original frame size
            cx, cy = int(x * width), int(y * height)
            # Draw a circle at the keypoint
            cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
    return frame

# Open video capture (use 0 for webcam; change to file path if needed)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame to get keypoints
    keypoints = process_frame(frame)
    # Draw keypoints on the frame
    frame_with_keypoints = draw_keypoints(frame, keypoints)

    cv2.imshow("Pose Estimation", frame_with_keypoints)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


/Users/ethankondev/Documents/COSI142/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to model files: /Users/ethankondev/.cache/kagglehub/models/google/movenet/tensorFlow2/singlepose-lightning/4


2025-03-14 12:38:08.657 Python[99152:1211863] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2025-03-14 12:38:09.899 Python[99152:1211863] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-14 12:38:09.899 Python[99152:1211863] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## Now with an exo skeleton

In [1]:
import cv2
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import kagglehub as kh

# Download the latest version of the model.
path = kh.model_download("google/movenet/tensorFlow2/singlepose-lightning")
print("Path to model files:", path)

# Load the model from the local path.
model = tf.saved_model.load(path)
movenet = model.signatures["serving_default"]

def process_frame(frame):
    # Resize frame to 192x192 (MoveNet’s expected input size)
    input_tensor = tf.image.resize(frame, (192, 192))
    input_tensor = tf.cast(input_tensor, dtype=tf.int32)
    input_tensor = tf.expand_dims(input_tensor, axis=0)
    outputs = model.signatures["serving_default"](input_tensor)
    # Output shape: [1, 1, 17, 3] => (y, x, confidence)
    keypoints = outputs["output_0"].numpy()
    return keypoints

def draw_skeleton(frame, keypoints, threshold=0.3):
    """
    Draws keypoints and connects them with lines to form a skeleton.
    Assumes keypoints are in normalized coordinates with shape (1, 1, 17, 3).
    """
    height, width, _ = frame.shape
    # Squeeze the output to shape (17, 3)
    kpts = keypoints[0, 0, :, :]

    # Define skeleton connections (based on MoveNet keypoint indices):
    skeleton_connections = [
        (0, 1), (0, 2),       # Nose to eyes
        (1, 3), (2, 4),       # Eyes to ears
        (0, 5), (0, 6),       # Nose to shoulders
        (5, 7), (7, 9),       # Left arm (shoulder -> elbow -> wrist)
        (6, 8), (8, 10),      # Right arm (shoulder -> elbow -> wrist)
        (5, 6),               # Shoulders
        (5, 11), (6, 12),     # Shoulders to hips
        (11, 12),             # Hips
        (11, 13), (13, 15),    # Left leg (hip -> knee -> ankle)
        (12, 14), (14, 16)     # Right leg (hip -> knee -> ankle)
    ]

    # First, compute absolute positions for each keypoint
    abs_kpts = []
    for kp in kpts:
        y, x, conf = kp
        abs_kpts.append((int(x * width), int(y * height), conf))
        # Draw the keypoint if confidence is high
        if conf > threshold:
            cv2.circle(frame, (int(x * width), int(y * height)), 5, (0, 255, 0), -1)

    # Draw the skeleton connections
    for connection in skeleton_connections:
        i, j = connection
        x1, y1, conf1 = abs_kpts[i]
        x2, y2, conf2 = abs_kpts[j]
        # Only draw the connection if both keypoints are confidently detected
        if conf1 > threshold and conf2 > threshold:
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    return frame

# Open video capture (0 for webcam; use file path for a video file)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    keypoints = process_frame(frame)
    # Draw skeleton on the frame
    frame_with_skeleton = draw_skeleton(frame, keypoints)

    cv2.imshow("Pose Estimation - Exo Skeleton", frame_with_skeleton)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


/Users/ethankondev/Documents/COSI142/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to model files: /Users/ethankondev/.cache/kagglehub/models/google/movenet/tensorFlow2/singlepose-lightning/4


2025-03-14 13:50:42.871 Python[99432:1224659] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2025-03-14 13:50:44.074 Python[99432:1224659] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-14 13:50:44.074 Python[99432:1224659] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## Push up form analyzer

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import kagglehub as kh
import math

# Define criteria for various workouts.
CRITERIA = {
    "Squat": [
        {
            "name": "knees",
            "description": "Knees reach 90 degrees",
            "long_description": "The angle between your hip, knee, and ankle must reach 90 degrees. Your thigh should be parallel to the ground.",
            "condition": lambda angles: angles["right_hipright_knee"] <= 90,
            "requirement": "require",
            "feedback_fail": "Lower your squat to at least 90 degrees!",
            "feedback_pass": "Great depth! Keep it up!",
        },
        {
            "name": "shoulders",
            "description": "Shoulders remain level",
            "long_description": "Your shoulders should be at the same height during the squat to avoid tilting.",
            "condition": lambda angles: angles["left_shoulderright_shoulder"] < 10,
            "requirement": "avoid",
            "feedback_fail": "Keep your shoulders level while squatting!",
            "feedback_pass": "Nice! Your shoulders stayed balanced!",
        },
        {
            "name": "torso",
            "description": "Torso remains upright",
            "long_description": "Your back should be straight, and eyes forward.",
            "condition": lambda angles: angles["right_shoulderright_hip"] >= 70,
            "requirement": "avoid",
            "feedback_fail": "Straighten your back and look forward!",
            "feedback_pass": "Great posture!",
        },
    ],
    "Push-Up": [
        {
            "name": "level-shoulders",
            "description": "Shoulders stay level",
            "long_description": "Your shoulders should not tilt past 25 degrees.",
            "condition": lambda angles: angles["left_shoulderright_shoulder"] < 25,
            "requirement": "avoid",
            "feedback_fail": "Keep your shoulders steady during push-ups!",
            "feedback_pass": "Good control over your shoulders!",
        },
        {
            "name": "tricep",
            "description": "Arms fully extend",
            "long_description": "At the top position, your triceps should form a 90-degree angle with the ground.",
            "condition": lambda angles: angles["right_shoulderright_elbow"] >= 80,
            "requirement": "require",
            "feedback_fail": "Extend your arms fully at the top!",
            "feedback_pass": "Good full extension!",
        },
    ],
    "Sit-Up": [
        {
            "name": "up-position",
            "description": "Back gets perpendicular to ground",
            "long_description": "When you are in the 'up' position, your back should be at least 75 degrees.",
            "condition": lambda angles: angles["right_shoulderright_hip"] >= 75,
            "requirement": "require",
            "feedback_fail": "Sit up higher to reach the correct position!",
            "feedback_pass": "Good sit-up position!",
        },
        {
            "name": "down-position",
            "description": "Back gets horizontal to ground",
            "long_description": "When you are in the down position, your back should be flat with the ground, including your neck.",
            "condition": lambda angles: angles["left_shoulderleft_hip"] <= 5,
            "requirement": "require",
            "feedback_fail": "Lie back fully to complete the rep!",
            "feedback_pass": "Nice full range of motion!",
        },
        {
            "name": "bent legs",
            "description": "Legs stay bent",
            "long_description": "During a sit-up, your legs should stay bent and not move excessively.",
            "condition": lambda angles: angles["right_hipright_knee"] <= 10,
            "requirement": "avoid",
            "feedback_fail": "Keep your legs stable and bent throughout the movement!",
            "feedback_pass": "Great job keeping your legs steady!",
        },
    ]
}

# Download and load the MoveNet model.
path = kh.model_download("google/movenet/tensorFlow2/singlepose-lightning")
print("Path to model files:", path)
model = tf.saved_model.load(path)
movenet = model.signatures["serving_default"]

def process_frame(frame):
    """Resizes the frame, runs inference with MoveNet, and returns keypoints."""
    input_tensor = tf.image.resize(frame, (192, 192))
    input_tensor = tf.cast(input_tensor, dtype=tf.int32)
    input_tensor = tf.expand_dims(input_tensor, axis=0)
    outputs = movenet(input_tensor)
    keypoints = outputs["output_0"].numpy()  # Shape: (1, 1, 17, 3)
    return keypoints

def draw_skeleton(frame, keypoints, threshold=0.3):
    """
    Draws keypoints and connects them with lines to form a skeleton.
    Assumes keypoints are in normalized coordinates with shape (1, 1, 17, 3).
    """
    height, width, _ = frame.shape
    # Squeeze the output to shape (17, 3)
    kpts = keypoints[0, 0, :, :]

    # Define skeleton connections (based on MoveNet keypoint indices):
    skeleton_connections = [
        (0, 1), (0, 2),       # Nose to eyes
        (1, 3), (2, 4),       # Eyes to ears
        (0, 5), (0, 6),       # Nose to shoulders
        (5, 7), (7, 9),       # Left arm (shoulder -> elbow -> wrist)
        (6, 8), (8, 10),      # Right arm (shoulder -> elbow -> wrist)
        (5, 6),               # Shoulders
        (5, 11), (6, 12),     # Shoulders to hips
        (11, 12),             # Hips
        (11, 13), (13, 15),    # Left leg (hip -> knee -> ankle)
        (12, 14), (14, 16)     # Right leg (hip -> knee -> ankle)
    ]

    # Compute absolute positions for each keypoint and draw circles.
    abs_kpts = []
    for kp in kpts:
        y, x, conf = kp
        abs_coord = (int(x * width), int(y * height), conf)
        abs_kpts.append(abs_coord)
        if conf > threshold:
            cv2.circle(frame, (int(x * width), int(y * height)), 5, (0, 255, 0), -1)

    # Draw the skeleton connections.
    for connection in skeleton_connections:
        i, j = connection
        x1, y1, conf1 = abs_kpts[i]
        x2, y2, conf2 = abs_kpts[j]
        if conf1 > threshold and conf2 > threshold:
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    return frame

def compute_pushup_angles(keypoints, frame_shape):
    """
    Compute angles required for analyzing push-up form:
      - The angle between the left and right shoulders (relative to horizontal).
      - The angle at the right elbow (formed by the right shoulder, elbow, and wrist).
    """
    height, width, _ = frame_shape
    kpts = keypoints[0, 0, :, :]  # Shape: (17, 3)

    # Convert normalized coordinates to absolute pixel values.
    def to_abs(kp):
        y, x, conf = kp
        return (int(x * width), int(y * height), conf)

    # Shoulder points: left (index 5) and right (index 6)
    left_shoulder = to_abs(kpts[5])
    right_shoulder = to_abs(kpts[6])
    dx = right_shoulder[0] - left_shoulder[0]
    dy = right_shoulder[1] - left_shoulder[1]
    # Compute the angle between the shoulder line and the horizontal.
    angle_shoulder = abs(math.degrees(math.atan2(dy, dx)))

    # Compute the right elbow angle using right shoulder (index 6), right elbow (index 8), and right wrist (index 10)
    def calculate_angle(a, b, c):
        ba = np.array([a[0] - b[0], a[1] - b[1]])
        bc = np.array([c[0] - b[0], c[1] - b[1]])
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
        return math.degrees(np.arccos(cosine_angle))

    right_elbow = to_abs(kpts[8])
    right_wrist = to_abs(kpts[10])
    right_shoulder_point = (right_shoulder[0], right_shoulder[1])
    right_elbow_point = (right_elbow[0], right_elbow[1])
    right_wrist_point = (right_wrist[0], right_wrist[1])
    angle_elbow = calculate_angle(right_shoulder_point, right_elbow_point, right_wrist_point)

    angles = {
        "left_shoulderright_shoulder": angle_shoulder,
        "right_shoulderright_elbow": angle_elbow
    }
    return angles

def analyze_pushup_form(angles, criteria):
    """
    Given computed angles and the push-up criteria, evaluate the form and return feedback messages.
    """
    feedback = []
    for criterion in criteria:
        if criterion["condition"](angles):
            feedback.append(criterion["feedback_pass"])
        else:
            feedback.append(criterion["feedback_fail"])
    return feedback

# Prompt the user to select the workout type.
workout = input("Enter workout type (Push-Up, Squat, Sit-Up): ").strip()
if workout not in CRITERIA:
    print("Workout type not recognized. Defaulting to Push-Up.")
    workout = "Push-Up"

window_name = "Workout Form Analysis - " + workout
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame to extract keypoints.
    keypoints = process_frame(frame)

    # Draw the skeleton on the frame.
    frame = draw_skeleton(frame, keypoints)

    # If push-up is selected, compute the angles and overlay feedback.
    if workout == "Push-Up":
        angles = compute_pushup_angles(keypoints, frame.shape)
        feedback_list = analyze_pushup_form(angles, CRITERIA["Push-Up"])
        # Display feedback messages.
        y0, dy = 30, 30
        for i, feedback in enumerate(feedback_list):
            y = y0 + i * dy
            cv2.putText(frame, feedback, (10, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    cv2.imshow(window_name, frame)

    # Quit if 'q' is pressed or if the window is closed.
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()


/Users/ethankondev/Documents/COSI142/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to model files: /Users/ethankondev/.cache/kagglehub/models/google/movenet/tensorFlow2/singlepose-lightning/4


2025-03-17 17:15:25.249 Python[17236:2491284] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2025-03-17 17:15:26.549 Python[17236:2491284] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-17 17:15:26.549 Python[17236:2491284] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 

## Push up form analyzer with recording

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import kagglehub as kh
import math

# Define joint labels based on MoveNet keypoint indices.
JOINT_LABELS = {
    0: "Nose",
    1: "Left Eye",
    2: "Right Eye",
    3: "Left Ear",
    4: "Right Ear",
    5: "Left Shoulder",
    6: "Right Shoulder",
    7: "Left Elbow",
    8: "Right Elbow",
    9: "Left Wrist",
    10: "Right Wrist",
    11: "Left Hip",
    12: "Right Hip",
    13: "Left Knee",
    14: "Right Knee",
    15: "Left Ankle",
    16: "Right Ankle"
}

# Define criteria for various workouts.
CRITERIA = {
    "Squat": [
        {
            "name": "knees",
            "description": "Knees reach 90 degrees",
            "long_description": "The angle between your hip, knee, and ankle must reach 90 degrees. Your thigh should be parallel to the ground.",
            "condition": lambda angles: angles["right_hipright_knee"] <= 90,
            "requirement": "require",
            "feedback_fail": "Lower your squat to at least 90 degrees!",
            "feedback_pass": "Great depth! Keep it up!",
        },
        {
            "name": "shoulders",
            "description": "Shoulders remain level",
            "long_description": "Your shoulders should be at the same height during the squat to avoid tilting.",
            "condition": lambda angles: angles["left_shoulderright_shoulder"] < 10,
            "requirement": "avoid",
            "feedback_fail": "Keep your shoulders level while squatting!",
            "feedback_pass": "Nice! Your shoulders stayed balanced!",
        },
        {
            "name": "torso",
            "description": "Torso remains upright",
            "long_description": "Your back should be straight, and eyes forward.",
            "condition": lambda angles: angles["right_shoulderright_hip"] >= 70,
            "requirement": "avoid",
            "feedback_fail": "Straighten your back and look forward!",
            "feedback_pass": "Great posture!",
        },
    ],
    "Push-Up": [
        {
            "name": "level-shoulders",
            "description": "Shoulders stay level",
            "long_description": "Your shoulders should not tilt past 25 degrees.",
            "condition": lambda angles: angles["left_shoulderright_shoulder"] < 25,
            "requirement": "avoid",
            "feedback_fail": "Keep your shoulders steady during push-ups!",
            "feedback_pass": "Good control over your shoulders!",
        },
        {
            "name": "tricep",
            "description": "Arms fully extend",
            "long_description": "At the top position, your triceps should form a 90-degree angle with the ground.",
            "condition": lambda angles: angles["right_shoulderright_elbow"] >= 80,
            "requirement": "require",
            "feedback_fail": "Extend your arms fully at the top!",
            "feedback_pass": "Good full extension!",
        },
    ],
    "Sit-Up": [
        {
            "name": "up-position",
            "description": "Back gets perpendicular to ground",
            "long_description": "When you are in the 'up' position, your back should be at least 75 degrees.",
            "condition": lambda angles: angles["right_shoulderright_hip"] >= 75,
            "requirement": "require",
            "feedback_fail": "Sit up higher to reach the correct position!",
            "feedback_pass": "Good sit-up position!",
        },
        {
            "name": "down-position",
            "description": "Back gets horizontal to ground",
            "long_description": "When you are in the down position, your back should be flat with the ground, including your neck.",
            "condition": lambda angles: angles["left_shoulderleft_hip"] <= 5,
            "requirement": "require",
            "feedback_fail": "Lie back fully to complete the rep!",
            "feedback_pass": "Nice full range of motion!",
        },
        {
            "name": "bent legs",
            "description": "Legs stay bent",
            "long_description": "During a sit-up, your legs should stay bent and not move excessively.",
            "condition": lambda angles: angles["right_hipright_knee"] <= 10,
            "requirement": "avoid",
            "feedback_fail": "Keep your legs stable and bent throughout the movement!",
            "feedback_pass": "Great job keeping your legs steady!",
        },
    ]
}

# Download and load the MoveNet model.
path = kh.model_download("google/movenet/tensorFlow2/singlepose-lightning")
print("Path to model files:", path)
model = tf.saved_model.load(path)
movenet = model.signatures["serving_default"]

def process_frame(frame):
    """Resizes the frame, runs inference with MoveNet, and returns keypoints."""
    input_tensor = tf.image.resize(frame, (192, 192))
    input_tensor = tf.cast(input_tensor, dtype=tf.int32)
    input_tensor = tf.expand_dims(input_tensor, axis=0)
    outputs = movenet(input_tensor)
    keypoints = outputs["output_0"].numpy()  # Shape: (1, 1, 17, 3)
    return keypoints

def draw_skeleton(frame, keypoints, threshold=0.3):
    """
    Draws keypoints and connects them with lines to form a skeleton.
    Labels are added at the midpoint of each connection.
    Assumes keypoints are in normalized coordinates with shape (1, 1, 17, 3).
    """
    height, width, _ = frame.shape
    kpts = keypoints[0, 0, :, :]  # Shape: (17, 3)

    # Define skeleton connections (based on MoveNet keypoint indices).
    skeleton_connections = [
        (0, 1), (0, 2),       # Nose to eyes
        (1, 3), (2, 4),       # Eyes to ears
        (0, 5), (0, 6),       # Nose to shoulders
        (5, 7), (7, 9),       # Left arm (shoulder -> elbow -> wrist)
        (6, 8), (8, 10),      # Right arm (shoulder -> elbow -> wrist)
        (5, 6),               # Shoulders
        (5, 11), (6, 12),     # Shoulders to hips
        (11, 12),             # Hips
        (11, 13), (13, 15),    # Left leg (hip -> knee -> ankle)
        (12, 14), (14, 16)     # Right leg (hip -> knee -> ankle)
    ]

    abs_kpts = []
    # Compute absolute coordinates and draw keypoint circles.
    for kp in kpts:
        y, x, conf = kp
        abs_coord = (int(x * width), int(y * height), conf)
        abs_kpts.append(abs_coord)
        if conf > threshold:
            cv2.circle(frame, (int(x * width), int(y * height)), 5, (0, 255, 0), -1)

    # Draw skeleton connections and add labels.
    for connection in skeleton_connections:
        i, j = connection
        x1, y1, conf1 = abs_kpts[i]
        x2, y2, conf2 = abs_kpts[j]
        if conf1 > threshold and conf2 > threshold:
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # Compute midpoint for label.
            mid_x = int((x1 + x2) / 2)
            mid_y = int((y1 + y2) / 2)
            label = f"{JOINT_LABELS[i]} - {JOINT_LABELS[j]}"
            cv2.putText(frame, label, (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (255, 255, 255), 1, cv2.LINE_AA)

    return frame

def compute_pushup_angles(keypoints, frame_shape):
    """
    Compute angles required for analyzing push-up form:
      - The angle between the left and right shoulders (relative to horizontal).
      - The angle at the right elbow (formed by the right shoulder, elbow, and wrist).
    """
    height, width, _ = frame_shape
    kpts = keypoints[0, 0, :, :]  # Shape: (17, 3)

    def to_abs(kp):
        y, x, conf = kp
        return (int(x * width), int(y * height), conf)

    left_shoulder = to_abs(kpts[5])
    right_shoulder = to_abs(kpts[6])
    dx = right_shoulder[0] - left_shoulder[0]
    dy = right_shoulder[1] - left_shoulder[1]
    angle_shoulder = abs(math.degrees(math.atan2(dy, dx)))

    def calculate_angle(a, b, c):
        ba = np.array([a[0] - b[0], a[1] - b[1]])
        bc = np.array([c[0] - b[0], c[1] - b[1]])
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
        return math.degrees(np.arccos(cosine_angle))

    right_elbow = to_abs(kpts[8])
    right_wrist = to_abs(kpts[10])
    right_shoulder_point = (right_shoulder[0], right_shoulder[1])
    right_elbow_point = (right_elbow[0], right_elbow[1])
    right_wrist_point = (right_wrist[0], right_wrist[1])
    angle_elbow = calculate_angle(right_shoulder_point, right_elbow_point, right_wrist_point)

    angles = {
        "left_shoulderright_shoulder": angle_shoulder,
        "right_shoulderright_elbow": angle_elbow
    }
    return angles

def analyze_pushup_form(angles, criteria):
    """
    Given computed angles and the push-up criteria, evaluate the form and return feedback messages.
    """
    feedback = []
    for criterion in criteria:
        if criterion["condition"](angles):
            feedback.append(criterion["feedback_pass"])
        else:
            feedback.append(criterion["feedback_fail"])
    return feedback

# Prompt the user to select the workout type.
workout = input("Enter workout type (Push-Up, Squat, Sit-Up): ").strip()
if workout not in CRITERIA:
    print("Workout type not recognized. Defaulting to Push-Up.")
    workout = "Push-Up"

window_name = "Workout Form Analysis - " + workout
cap = cv2.VideoCapture(0)

# Set up video recording using properties from the webcam.
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 30  # Fallback if FPS cannot be detected.
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame and extract keypoints.
    keypoints = process_frame(frame)

    # Draw the skeleton (with connection labels).
    frame = draw_skeleton(frame, keypoints)

    # For push-ups, compute angles and overlay feedback.
    if workout == "Push-Up":
        angles = compute_pushup_angles(keypoints, frame.shape)
        feedback_list = analyze_pushup_form(angles, CRITERIA["Push-Up"])
        y0, dy = 30, 30
        for i, feedback in enumerate(feedback_list):
            y = y0 + i * dy
            cv2.putText(frame, feedback, (10, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Write the processed frame to the output video.
    out.write(frame)
    cv2.imshow(window_name, frame)

    # Exit if 'q' is pressed or if the window is closed.
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


/Users/ethankondev/Documents/COSI142/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to model files: /Users/ethankondev/.cache/kagglehub/models/google/movenet/tensorFlow2/singlepose-lightning/4


2025-03-16 17:46:36.230 Python[9558:1975110] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2025-03-16 17:46:37.435 Python[9558:1975110] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-16 17:46:37.435 Python[9558:1975110] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 

# More Push-Up Form Criteria

In [1]:
import cv2
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import kagglehub as kh
import math

# Define joint labels based on MoveNet keypoint indices.
JOINT_LABELS = {
    0: "Nose",
    1: "Left Eye",
    2: "Right Eye",
    3: "Left Ear",
    4: "Right Ear",
    5: "Left Shoulder",
    6: "Right Shoulder",
    7: "Left Elbow",
    8: "Right Elbow",
    9: "Left Wrist",
    10: "Right Wrist",
    11: "Left Hip",
    12: "Right Hip",
    13: "Left Knee",
    14: "Right Knee",
    15: "Left Ankle",
    16: "Right Ankle"
}

# Define criteria for various workouts.
CRITERIA = {
    "Squat": [
        {
            "name": "knees",
            "description": "Knees reach 90 degrees",
            "long_description": "The angle between your hip, knee, and ankle must reach 90 degrees. Your thigh should be parallel to the ground.",
            "condition": lambda angles: angles["right_hipright_knee"] <= 90,
            "requirement": "require",
            "feedback_fail": "Lower your squat to at least 90 degrees!",
            "feedback_pass": "Great depth! Keep it up!",
        },
        {
            "name": "shoulders",
            "description": "Shoulders remain level",
            "long_description": "Your shoulders should be at the same height during the squat to avoid tilting.",
            "condition": lambda angles: angles["left_shoulderright_shoulder"] < 10,
            "requirement": "avoid",
            "feedback_fail": "Keep your shoulders level while squatting!",
            "feedback_pass": "Nice! Your shoulders stayed balanced!",
        },
        {
            "name": "torso",
            "description": "Torso remains upright",
            "long_description": "Your back should be straight, and eyes forward.",
            "condition": lambda angles: angles["right_shoulderright_hip"] >= 70,
            "requirement": "avoid",
            "feedback_fail": "Straighten your back and look forward!",
            "feedback_pass": "Great posture!",
        },
    ],
    "Push-Up": [
        {
            "name": "level-shoulders",
            "description": "Shoulders stay level",
            "long_description": "Your shoulders should not tilt past 25 degrees.",
            "condition": lambda angles: angles["left_shoulderright_shoulder"] < 25,
            "requirement": "avoid",
            "feedback_fail": "Keep your shoulders steady during push-ups!",
            "feedback_pass": "Good control over your shoulders!",
        },
        {
            "name": "tricep",
            "description": "Arms fully extend",
            "long_description": "At the top position, your triceps should form a 90-degree angle with the ground.",
            "condition": lambda angles: angles["right_shoulderright_elbow"] >= 80,
            "requirement": "require",
            "feedback_fail": "Extend your arms fully at the top!",
            "feedback_pass": "Good full extension!",
        },
        {
            "name": "body-alignment",
            "description": "Body remains straight",
            "long_description": "Your body should form a straight line from your shoulder to your hip.",
            "condition": lambda angles: angles["body_alignment"] < 10,
            "requirement": "require",
            "feedback_fail": "Keep your core engaged and maintain a straight body!",
            "feedback_pass": "Great body alignment!",
        },
        {
            "name": "elbow-flare",
            "description": "Elbows remain tucked",
            "long_description": "Your elbows should stay close to your midline instead of flaring out.",
            "condition": lambda angles: angles["elbow_offset_ratio"] < 0.3,
            "requirement": "avoid",
            "feedback_fail": "Tuck your elbows in closer to your body!",
            "feedback_pass": "Good elbow position!",
        },
    ],
    "Sit-Up": [
        {
            "name": "up-position",
            "description": "Back gets perpendicular to ground",
            "long_description": "When you are in the 'up' position, your back should be at least 75 degrees.",
            "condition": lambda angles: angles["right_shoulderright_hip"] >= 75,
            "requirement": "require",
            "feedback_fail": "Sit up higher to reach the correct position!",
            "feedback_pass": "Good sit-up position!",
        },
        {
            "name": "down-position",
            "description": "Back gets horizontal to ground",
            "long_description": "When you are in the down position, your back should be flat with the ground, including your neck.",
            "condition": lambda angles: angles["left_shoulderleft_hip"] <= 5,
            "requirement": "require",
            "feedback_fail": "Lie back fully to complete the rep!",
            "feedback_pass": "Nice full range of motion!",
        },
        {
            "name": "bent legs",
            "description": "Legs stay bent",
            "long_description": "During a sit-up, your legs should stay bent and not move excessively.",
            "condition": lambda angles: angles["right_hipright_knee"] <= 10,
            "requirement": "avoid",
            "feedback_fail": "Keep your legs stable and bent throughout the movement!",
            "feedback_pass": "Great job keeping your legs steady!",
        },
    ]
}

# Download and load the MoveNet model.
path = kh.model_download("google/movenet/tensorFlow2/singlepose-lightning")
print("Path to model files:", path)
model = tf.saved_model.load(path)
movenet = model.signatures["serving_default"]

def process_frame(frame):
    """Resizes the frame, runs inference with MoveNet, and returns keypoints."""
    input_tensor = tf.image.resize(frame, (192, 192))
    input_tensor = tf.cast(input_tensor, dtype=tf.int32)
    input_tensor = tf.expand_dims(input_tensor, axis=0)
    outputs = movenet(input_tensor)
    keypoints = outputs["output_0"].numpy()  # Shape: (1, 1, 17, 3)
    return keypoints

def draw_skeleton(frame, keypoints, threshold=0.3):
    """
    Draws keypoints and connects them with lines to form a skeleton.
    Labels are added at the midpoint of each connection.
    Assumes keypoints are in normalized coordinates with shape (1, 1, 17, 3).
    """
    height, width, _ = frame.shape
    kpts = keypoints[0, 0, :, :]  # Shape: (17, 3)

    # Define skeleton connections (based on MoveNet keypoint indices).
    skeleton_connections = [
        (0, 1), (0, 2),       # Nose to eyes
        (1, 3), (2, 4),       # Eyes to ears
        (0, 5), (0, 6),       # Nose to shoulders
        (5, 7), (7, 9),       # Left arm (shoulder -> elbow -> wrist)
        (6, 8), (8, 10),      # Right arm (shoulder -> elbow -> wrist)
        (5, 6),               # Shoulders
        (5, 11), (6, 12),     # Shoulders to hips
        (11, 12),             # Hips
        (11, 13), (13, 15),    # Left leg (hip -> knee -> ankle)
        (12, 14), (14, 16)     # Right leg (hip -> knee -> ankle)
    ]

    abs_kpts = []
    # Compute absolute coordinates and draw keypoint circles.
    for kp in kpts:
        y, x, conf = kp
        abs_coord = (int(x * width), int(y * height), conf)
        abs_kpts.append(abs_coord)
        if conf > threshold:
            cv2.circle(frame, (int(x * width), int(y * height)), 5, (0, 255, 0), -1)

    # Draw skeleton connections and add labels.
    for connection in skeleton_connections:
        i, j = connection
        x1, y1, conf1 = abs_kpts[i]
        x2, y2, conf2 = abs_kpts[j]
        if conf1 > threshold and conf2 > threshold:
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # Compute midpoint for label.
            mid_x = int((x1 + x2) / 2)
            mid_y = int((y1 + y2) / 2)
            label = f"{JOINT_LABELS[i]} - {JOINT_LABELS[j]}"
            cv2.putText(frame, label, (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (255, 255, 255), 1, cv2.LINE_AA)
    return frame

def compute_pushup_angles(keypoints, frame_shape):
    """
    Compute angles required for analyzing push-up form:
      - The angle between the left and right shoulders.
      - The angle at the right elbow.
      - The body alignment angle (right shoulder to right hip relative to horizontal).
      - The ratio of the right elbow's horizontal offset from the body's midline.
    """
    height, width, _ = frame_shape
    kpts = keypoints[0, 0, :, :]  # Shape: (17, 3)

    def to_abs(kp):
        y, x, conf = kp
        return (int(x * width), int(y * height), conf)

    # Shoulders for level check.
    left_shoulder = to_abs(kpts[5])
    right_shoulder = to_abs(kpts[6])
    dx_shoulder = right_shoulder[0] - left_shoulder[0]
    dy_shoulder = right_shoulder[1] - left_shoulder[1]
    angle_shoulder = abs(math.degrees(math.atan2(dy_shoulder, dx_shoulder)))
    
    # Compute the right elbow angle.
    def calculate_angle(a, b, c):
        ba = np.array([a[0] - b[0], a[1] - b[1]])
        bc = np.array([c[0] - b[0], c[1] - b[1]])
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
        return math.degrees(np.arccos(cosine_angle))
    
    right_elbow = to_abs(kpts[8])
    right_wrist = to_abs(kpts[10])
    right_shoulder_point = (right_shoulder[0], right_shoulder[1])
    right_elbow_point = (right_elbow[0], right_elbow[1])
    right_wrist_point = (right_wrist[0], right_wrist[1])
    angle_elbow = calculate_angle(right_shoulder_point, right_elbow_point, right_wrist_point)
    
    # Compute body alignment: angle between right shoulder and right hip.
    right_hip = to_abs(kpts[12])
    dx_body = right_hip[0] - right_shoulder[0]
    dy_body = right_hip[1] - right_shoulder[1]
    angle_body_alignment = abs(math.degrees(math.atan2(dy_body, dx_body)))
    
    # Compute elbow offset from the body's midline.
    left_hip = to_abs(kpts[11])
    mid_shoulder_x = (left_shoulder[0] + right_shoulder[0]) / 2
    mid_hip_x = (left_hip[0] + right_hip[0]) / 2
    midline_x = (mid_shoulder_x + mid_hip_x) / 2
    elbow_offset = abs(right_elbow[0] - midline_x)
    dist_shoulder_hip = math.sqrt((right_hip[0] - right_shoulder[0])**2 + (right_hip[1] - right_shoulder[1])**2)
    elbow_offset_ratio = elbow_offset / (dist_shoulder_hip + 1e-6)
    
    angles = {
        "left_shoulderright_shoulder": angle_shoulder,
        "right_shoulderright_elbow": angle_elbow,
        "body_alignment": angle_body_alignment,
        "elbow_offset_ratio": elbow_offset_ratio
    }
    return angles

def analyze_pushup_form(angles, criteria):
    """
    Given computed angles and the push-up criteria, evaluate the form and return feedback messages.
    """
    feedback = []
    for criterion in criteria:
        if criterion["condition"](angles):
            feedback.append(criterion["feedback_pass"])
        else:
            feedback.append(criterion["feedback_fail"])
    return feedback

# Prompt the user to select the workout type.
workout = input("Enter workout type (Push-Up, Squat, Sit-Up): ").strip()
if workout not in CRITERIA:
    print("Workout type not recognized. Defaulting to Push-Up.")
    workout = "Push-Up"

window_name = "Workout Form Analysis - " + workout
cap = cv2.VideoCapture(0)

# Set up video recording using properties from the webcam.
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 30  # Fallback if FPS cannot be detected.
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_test.avi', fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame and extract keypoints.
    keypoints = process_frame(frame)
    # Draw the skeleton (with connection labels).
    frame = draw_skeleton(frame, keypoints)

    # For push-ups, compute angles and overlay feedback.
    if workout == "Push-Up":
        angles = compute_pushup_angles(keypoints, frame.shape)
        feedback_list = analyze_pushup_form(angles, CRITERIA["Push-Up"])
        y0, dy = 30, 30
        for i, feedback in enumerate(feedback_list):
            y = y0 + i * dy
            cv2.putText(frame, feedback, (10, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Write the processed frame to the output video.
    out.write(frame)
    cv2.imshow(window_name, frame)

    # Exit if 'q' is pressed or if the window is closed.
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


/Users/ethankondev/Documents/COSI142/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to model files: /Users/ethankondev/.cache/kagglehub/models/google/movenet/tensorFlow2/singlepose-lightning/4


2025-03-23 13:48:10.417 Python[32013:5955758] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-23 13:48:10.417 Python[32013:5955758] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [8]:
def calculate_squat(kps):
'''0: nose
1: left_eye
2: right_eye
3: left_ear
4: right_ear
5: left_shoulder
6: right_shoulder
7: left_elbow
8: right_elbow
9: left_wrist
10: right_wrist
11: left_hip
12: right_hip
13: left_knee
14: right_knee
15: left_ankle
16: right_ankle '''
    right_hip = kps[12]
    right_knee = kps[14]
    

IndentationError: expected an indented block (365411458.py, line 2)

In [ ]:
def evaluate_pose(pose_name, angles):
    """Evaluates a user's movement against predefined criteria."""
    results = []
    feedback = []

    if pose_name not in CRITERIA:
        return {"error": "Pose not found"}

    for criterion in CRITERIA[pose_name]:
        is_valid = criterion["condition"](angles)
        results.append((criterion["name"], is_valid))

        # Add feedback
        feedback.append(
            f"{criterion['description']}: " +
            (criterion["feedback_pass"] if is_valid else criterion["feedback_fail"])
        )

    return {"evaluation": results, "feedback": feedback}
